In [ ]:
import constants as const
import numpy as np
%matplotlib nbagg
import matplotlib
from matplotlib.pyplot import *
from math import exp, atan, sqrt, fabs
from scipy.integrate import simps, odeint, quad
from scipy.optimize import brentq, root, fsolve

import gas_sphere
import blackbody_emitter


#### Main inputs ####

# Emitter properties (this test problem uses a monochromatic emitter)
emission_frequency = 4.32e15 # Hz
Q = 5.e48 # photons / second

# Atomic level information, for pure H
n_levels = 3
no_ground_recomb = 0

# Zone information
density = 1.6726e-27 # g/cm^3
r = 1.0e22 # zone position (cm)
r0 = 9.99e21 # Emitter photosphere (cm)




In [ ]:
# Just run this cell (will take a few seconds to complete)

#### Derived parameters and constants ####

nu_0 = 3.28e15 # Ionization threshold of ground-state hydrogen (Hz)
delta_nu = emission_frequency - nu_0
sigma = 6.3e-18 * pow((nu_0 + delta_nu)/nu_0,-3.) # photoionization cross section for ground state hydrogen at the specified emission frequency
Z = 1 # atomic number of H
n_H = density/const.m_p
n_e = n_H
L = Q * const.h * (nu_0 + delta_nu) 
W = 0.5*(1. - (1. - r0**2/r**2)**0.5)
J = L*W/(4.0*np.pi**2*r0**2)

 
# Frequency grid setup: meant to replicate the frequency grid in the param file
shortest_wavelength = const.c/(3.e19 * const.angs_to_cm )
longest_wavelength = const.c/(3.5e11 * const.angs_to_cm )
num_wavelengths = 9143


# Electron temperatures for plotting heating/
Te_grid = np.logspace(1.,9.,128)



#### Set up data structures ####

gas = gas_sphere.GasSphere()                                                                                                                                                                                                            
bb = blackbody_emitter.BlackbodyEmitter()


bb.shortest_wavelength_angs = shortest_wavelength
bb.longest_wavelength_angs = longest_wavelength
bb.num_wavelengths = num_wavelengths
bb.num_frequencies = num_wavelengths
bb.reset_wavelength_grid()
bb.reset_frequency_grid()


#### Compute H recombination coefficients ####

H_level_energy_list = np.array([])
H_level_degeneracy_list = np.array([])

for n_quantum in range(1,1 + n_levels):
    H_level_energy_list = np.append(H_level_energy_list, 13.6 * (1. - 1./pow(n_quantum,2.)))
    H_level_degeneracy_list = np.append(H_level_degeneracy_list,2. * pow(n_quantum,2.))
    
print "Atomic energy levels are (eV): ", H_level_energy_list
print "Level statistical weights are: ", H_level_degeneracy_list
                                    

gas.add_element(1,1.,H_level_energy_list,H_level_degeneracy_list)

H = gas.elements[0]

alpha_Bs = np.zeros_like(Te_grid)
alpha_As = np.zeros_like(Te_grid)

print "Computing recombination coefficients..."

for T_index in range(len(Te_grid)):
    
    Te = Te_grid[T_index]

    alpha_A_hydrogen = 0.
    for i in range(H.num_levels):
        H.levels[i].alpha = H.compute_alpha(i,Te,bb)
        alpha_A_hydrogen +=  H.levels[i].alpha
        
    
    alpha_Bs[T_index] = alpha_A_hydrogen - H.compute_alpha(0,Te,bb)
    alpha_As[T_index] = alpha_A_hydrogen 
    
    
print "Finished computing recombination coefficients"

#### Define other opacities ####
    
def freefree_alpha_nu(this_nu,ne,ni,T):
    
    return 0.0177 * pow(T,-1.5) * pow(Z,2.) * ne * ni * pow(this_nu,-2.)

def freefree_jnu(this_nu,ne,ni,T):
    
    return 6.8e-38/(4. * np.pi) * pow(T,-0.5) * pow(Z,2.) * ne * ni * exp(-const.h * this_nu/(const.k * T))





In [ ]:
# Just run this cell (will take a few seconds to complete)

### Compute heating and cooling rates ####


def bf_heating_rate(this_T):
    
    T_e = this_T
    
    alpha_A_hydrogen = 0.
    for i in range(H.num_levels):
        H.levels[i].alpha = H.compute_alpha(i,T_e,bb)
        alpha_A_hydrogen +=  H.levels[i].alpha
        
    
    alpha_B = alpha_A_hydrogen - H.compute_alpha(0,T_e,bb)
    
    
    if (no_ground_recomb == 1):
        n_ground = n_e * n_e * alpha_B * const.h * (nu_0 + delta_nu)/(4. * np.pi * J * sigma )
    else:
        n_ground = n_e * n_e * alpha_A_hydrogen * const.h * (nu_0 + delta_nu)/(4. * np.pi * J * sigma )
    
    return n_ground * sigma * 4 * np.pi * J/(const.h * (nu_0 + delta_nu))  * const.h * delta_nu 


def bf_cooling_rate(this_T):
    
    T_e = this_T
    
    bf_cooling_rate_integrand = np.zeros(bb.num_frequencies)
    
    if (no_ground_recomb == 1):
        starting_level_index = 1
    else:
        starting_level_index = 0
    
    for l in range(starting_level_index, H.num_levels):  # if you're working in the case B limit, don't include cooling from recombinations to ground state
    
        this_level = H.levels[l]

        photoion_cs = np.array([H.approx_photoion_cs(nu,l,this_level.n_quantum) for nu in bb.frequencies])
    
        nc = n_e # special case of pure hydrogen
        gc = 1.
        gl = H.levels[l].statistical_weight
    
        E_ion = const.h * H.levels[l].ionization_frequency_threshold
    
        for i in range(bb.num_frequencies):

            nu = bb.frequencies[i]
        
            E = const.h * nu
              
            if (E < E_ion):
                bf_cooling_rate_integrand[i] += 0.
            else:
                
                bf_cooling_rate_integrand[i] += 4. * np.pi * n_e * nc * photoion_cs[i] * (E - E_ion)/E * 2. * const.h * nu**3. /const.c**2. * pow(const.h**2./(2. * np.pi * const.m_e * const.k * T_e),3./2.) * gl/(2. * gc) * exp( -1. * (E - E_ion)/(const.k * T_e) )
        

    return simps(bf_cooling_rate_integrand,bb.frequencies)


def ff_heating_rate(this_T):
    
    T_e = this_T
    
    ff_heating_rate_integrand = np.zeros(bb.num_frequencies)
    
    for i in range(bb.num_frequencies):

        nu = bb.frequencies[i]
        ff_heating_rate_integrand[i] = 4 * np.pi * J * freefree_alpha_nu(nu_0 + delta_nu,n_e,n_e,T_e)
   
    return simps(ff_heating_rate_integrand,bb.frequencies)


def ff_cooling_rate(this_T):
    
    T_e = this_T
    
    ff_cooling_rate_integrand = np.zeros(bb.num_frequencies)
    
    for i in range(bb.num_frequencies):

        nu = bb.frequencies[i]           
        ff_cooling_rate_integrand[i] = 4. * np.pi * freefree_jnu(nu,n_e, n_e, T_e)
        

    return simps(ff_cooling_rate_integrand,bb.frequencies)    


def radiative_equilibrium(this_T):
    
    return bf_heating_rate(this_T) + ff_heating_rate(this_T) - bf_cooling_rate(this_T) - ff_cooling_rate(this_T)



########

print "Computing heating & cooling rates"

bf_heating_rates = np.array([bf_heating_rate(Te) for Te in Te_grid])
bf_cooling_rates = np.array([bf_cooling_rate(Te) for Te in Te_grid])
ff_heating_rates = np.array([ff_heating_rate(Te) for Te in Te_grid])
ff_cooling_rates = np.array([ff_cooling_rate(Te) for Te in Te_grid])



    
print "Finished computing heating & cooling rates"

In [ ]:
# Just run this cell (will take a few seconds)

close()

temperature_solution = fsolve(radiative_equilibrium, 1.e4)[0]

print "Equilibrium temperature should be ", temperature_solution
print "\n"
print "This is where total heating rate (black dashed curve) intersects total cooling rate (black solid curve)"
print "Cooling rate contains contributions from bound-free (blue) and free-free (red)"
print "Free-free heating contribution is usually negligible"


loglog(Te_grid,bf_cooling_rates,'-',c='b')
loglog(Te_grid,ff_cooling_rates,'-',c='r')
loglog(Te_grid,bf_cooling_rates + ff_cooling_rates,'-',c='k')
loglog(Te_grid, bf_heating_rates + ff_heating_rates,'--',c='k')

xlabel("temperature (K)")
ylabel("Rate (erg / s /cm^3)")


